#Predicción de calificaciones

Cuando diseñamos e implementamos un curso en cualquier área del conocimiento, es fundamental contar con una perspectiva tanto cuantitativa como cualitativa. En particular, los datos cuantitativos permiten evaluar numéricamente el aprendizaje de las y los estudiantes, lo cual resulta clave para determinar si las estrategias empleadas están contribuyendo al logro de los objetivos propuestos.

A través de actividades evaluables a las que se les pueda asignar una calificación numérica —como exámenes, ejercicios, tareas o proyectos— es posible registrar y analizar el progreso del grupo. Esta información resulta valiosa para valorar la efectividad del curso y tomar decisiones informadas sobre su desarrollo y posibles mejoras.

Como complemento a este enfoque cuantitativo, buscamos implementar una red neuronal que nos permita predecir las calificaciones de las actividades de las y los estudiantes. Una vez aplicada esta estrategia, dispondremos de una base de datos con la que podremos realizar predicciones sobre el desempeño futuro en el curso.

Por ejemplo, si un curso consta de 5 actividades y un estudiante ha completado 3, el modelo buscará predecir las calificaciones que podría obtener en las 2 actividades restantes. Esto no solo contribuirá al seguimiento individualizado del aprendizaje, sino que también permitirá identificar patrones y ajustar la enseñanza de manera más precisa y anticipada.

Usaremos una base de datos donde tienen distintos parámetros y buscamos predecir la calificación final, esta base de datos la obtuvimos en https://www.kaggle.com/datasets

In [1]:
# Primero importamos Tensorflow, Keras y las librerías que usaremos
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

import numpy as np
import pandas as pd

import random
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Buscamos predecir la calificación de las actividades 3 y 4 dado que un estudiante ya tiene las actividades 1, 2 y 3

In [4]:
# Cargamos los datos desde el archivo que ya tiene formato csv
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datos_calificaciones.csv')
data.head()

Mounted at /content/drive


,ID_Alumno,Calificacion_Actividad_1,Calificacion_Actividad_2,Calificacion_Actividad_3,Calificacion_Actividad_4,Calificacion_Actividad_5
0,1,7.5,8.5,6.0,6.0,10.0
1,2,7.0,10.0,10.0,10.0,9.5
2,3,5.0,8.0,10.0,7.0,10.0
3,4,10.5,9.0,10.0,10.0,8.5
4,5,5.0,7.5,10.0,6.5,5.0


In [5]:
#Separamos el conjunto de datos original en dos partes:
#X que son las variables independientes que se utilizarán para predecir las y, que son las variables dependientes que se pretenden predecir o modelar.
X = data[['Calificacion_Actividad_1', 'Calificacion_Actividad_2', 'Calificacion_Actividad_3', 'Calificacion_Actividad_4', 'Calificacion_Actividad_5']]
y = data[['Calificacion_Actividad_4', 'Calificacion_Actividad_5']]

#Divide los datos en conjuntos de entrenamiento y prueba
#El 20% de los datos se utilizarán para el conjunto de prueba, y el 80% se utilizarán para el conjunto de entrenamiento.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#Creamos el modelo de la red neuronal
#Será un modelo secuencial con 3 capas.
#La primera capa es oculta y tiene 10 neuronas con una función de activación ReLU
#La segunda capa es de salida y tiene 2 neuronas (actividades 4 y 5)
#Este modelo puede utilizarse para resolver problemas de regresión, donde se busca predecir valores numéricos en lugar de clases o categorías.
model = Sequential()
model.add(Dense(10, input_dim=5, activation='relu'))
model.add(Dense(2, activation='relu'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
#Se usará MeanSquaredError para calcular el error de la predicción
#Se usará "adam" para sustituir el descenso del gradiente y la métrica de evaluación como error absoluto medio.
#Estas opciones permiten entrenar el modelo para minimizar la pérdida y evaluar su rendimiento en términos de precisión de predicción
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

In [8]:
# Veamos el resumen del modelo para comprobar que tenemos lo deseado
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 82 (328.00 B)

 Trainable params: 82 (328.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
#Realizamos el entrenamiento del modelo utilizando los conjuntos de entrenamiento y prueba
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - loss: 28.9502 - mae: 3.9743 - val_loss: 33.0874 - val_mae: 4.4002
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 27.7601 - mae: 3.8336 - val_loss: 32.9894 - val_mae: 4.3853
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 27.9511 - mae: 3.8485 - val_loss: 32.9141 - val_mae: 4.3725
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 29.4803 - mae: 3.9572 - val_loss: 32.8612 - val_mae: 4.3608
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 29.1472 - mae: 3.9423 - val_loss: 32.8218 - val_mae: 4.3498
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 27.8918 - mae: 3.8190 - val_loss: 32.7899 - val_mae: 4.3402
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 28.0115 - mae: 3.8040 - val_loss: 32.7602 - val_mae: 4.3307
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 28.6329 - mae: 3.8632 - val_loss: 32.7332 - val_mae: 4.3216
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - los

In [10]:
#Realizamos la evaluación del modelo en el conjunto de prueba y muestra los valores de la función de pérdida (loss) y el error absoluto medio (MAE).
#Estos valores proporcionan una medida del rendimiento del modelo en términos de la precisión de las predicciones en el conjunto de prueba.
#Cuanto menor sea la función de pérdida y el MAE, mejor será el rendimiento del modelo en la tarea de predecir las calificaciones de las últimas 2 actividades.
loss, mae = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("MAE:", mae)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.4552 - mae: 0.5197
Loss: 0.45519009232521057
MAE: 0.5196605920791626


Para poner en marcha el modelo generamos un dataset de 100 registros de estas calificaciones aleatorias que correspondan a la actividad 3 y 4

In [11]:
#Cargamos el archivo de esas nuevas calificaciones ficticias
new_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/new_data2.csv')
new_data.head()

,ID_Alumno,Calificacion_Actividad_1,Calificacion_Actividad_2,Calificacion_Actividad_3,Calificacion_Actividad_4,Calificacion_Actividad_5
0,1,3.7,0.3,6.4,0,0
1,2,9.5,6.4,0.8,0,0
2,3,7.3,3.1,1.6,0,0
3,4,6.0,5.1,9.0,0,0
4,5,1.6,9.1,6.1,0,0


In [12]:
#Obtemos las primeras 3 calificaciones de los nuevos estudiantes
X_new = new_data[['Calificacion_Actividad_1', 'Calificacion_Actividad_2', 'Calificacion_Actividad_3']]

#Agregamos columnas adicionales con valores (0.0) para las características faltantes
X_new_modified = np.concatenate((X_new, np.zeros((X_new.shape[0], 2))), axis=1)

#Realiza las predicciones para las próximas 2 calificaciones
predictions = model.predict(X_new_modified)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [13]:
predictions

array([[0.        , 0.21778384],
       [0.8412488 , 0.        ],
       [0.        , 0.20667973],
       [0.        , 0.09518281],
       [0.        , 0.        ],
       [0.4165245 , 0.        ],
       [0.3162475 , 0.        ],
       [0.        , 0.        ],
       [0.1411005 , 0.41078034],
       [0.        , 0.71245396],
       [0.        , 0.        ],
       [0.        , 0.7660221 ],
       [0.4169809 , 0.        ],
       [0.77022904, 0.        ],
       [0.        , 0.        ],
       [0.7365127 , 0.        ],
       [0.58728796, 0.        ],
       [0.        , 0.38471976],
       [0.        , 0.        ],
       [0.        , 0.00779501],
       [0.16583768, 0.        ],
       [0.        , 0.        ],
       [0.35888728, 0.        ],
       [0.        , 0.30852094],
       [0.        , 0.13068047],
       [0.07336489, 0.0146617 ],
       [0.        , 0.        ],
       [0.8700234 , 0.        ],
       [0.        , 0.42561188],
       [0.        , 0.        ],
       [0.

Sin embargo, son muy pocos los datos con los que contamos. Así que ahora crearemos una base de datos con muchos registros tomando como referencia los  datos que ya tenemos.

In [14]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.0 MB/s eta 0:00:00


In [18]:
#Para esto me ayudó chat gpt
from faker import Faker

#Cargamos la base de datos existente
existing_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datos_calificaciones.csv')

#Creamos un generador de datos sintéticos
fake = Faker()

#Definimos el número de registros sintéticos que deseas generar
num_records = 1000

#Generamos registros sintéticos
synthetic_data = pd.DataFrame(columns=existing_data.columns)  #Crear un DataFrame vacío con las mismas columnas que la base de datos existente

for _ in range(num_records):
    synthetic_row = {}
    for column in existing_data.columns:
        if column == 'ID':
            synthetic_row[column] = fake.uuid4()  # Generar un ID único con faker
        else:
            synthetic_row[column] = fake.random_element(existing_data[column])  # Generar valores aleatorios basados en los datos existentes
    synthetic_data = pd.concat([synthetic_data, pd.DataFrame([synthetic_row])], ignore_index=True)  # Agregar el registro sintético al DataFrame

# Combinar la base de datos existente con los registros sintéticos
combined_data = pd.concat([existing_data, synthetic_data], ignore_index=True)

# Guardar la base de datos combinada en un nuevo archivo CSV
combined_data.to_csv('nueva_base_de_datos.csv', index=False)

<ipython-input-18-164daac7c0f3>:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  synthetic_data = pd.concat([synthetic_data, pd.DataFrame([synthetic_row])], ignore_index=True)  # Agregar el registro sintético al DataFrame


Hacemos nuevamente todo el procedimiento

In [19]:
df = pd.read_csv('/content/nueva_base_de_datos.csv')
df.head()

,ID_Alumno,Calificacion_Actividad_1,Calificacion_Actividad_2,Calificacion_Actividad_3,Calificacion_Actividad_4,Calificacion_Actividad_5
0,1,7.5,8.5,6.0,6.0,10.0
1,2,7.0,10.0,10.0,10.0,9.5
2,3,5.0,8.0,10.0,7.0,10.0
3,4,10.5,9.0,10.0,10.0,8.5
4,5,5.0,7.5,10.0,6.5,5.0


In [20]:
X = df[['Calificacion_Actividad_1', 'Calificacion_Actividad_2', 'Calificacion_Actividad_3', 'Calificacion_Actividad_4', 'Calificacion_Actividad_5']]
y = df[['Calificacion_Actividad_4', 'Calificacion_Actividad_5']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X_train.shape[0]

868

In [22]:
modelo = Sequential()
modelo.add(Dense(10, input_dim=5, activation='relu'))
modelo.add(Dense(2, activation='relu'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
#modelo.summary()

In [24]:
modelo.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 32.7094 - mae: 4.5520 - val_loss: 32.8801 - val_mae: 4.6173
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 32.4991 - mae: 4.5477 - val_loss: 32.0649 - val_mae: 4.5414
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30.1609 - mae: 4.3127 - val_loss: 31.4162 - val_mae: 4.4656
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30.0534 - mae: 4.2906 - val_loss: 30.8046 - val_mae: 4.3958
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30.0038 - mae: 4.2459 - val_loss: 30.2383 - val_mae: 4.3258
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.5337 - mae: 4.1732 - val_loss: 29.7484 - val_mae: 4.2580
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 28.1278 - mae: 4.0103 - val_loss: 29.2660 - val_mae: 4.1867
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 28.4223 - mae: 4.0455 - val_loss: 28.8389 - val_mae: 4.1239
Epoch 9/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/s

In [25]:
loss, mae = modelo.evaluate(X_test, y_test)
print("Loss:", loss)
print("MAE:", mae)
#Aquí fue menor la función de pérdida y el mae, pero usamos más epocas

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 25.8102 - mae: 3.1924 
Loss: 25.460458755493164
MAE: 3.162416696548462


In [26]:
predictions = modelo.predict(X_new_modified)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [27]:
predictions

array([[0.        , 0.11405805],
       [0.        , 0.3471782 ],
       [0.        , 0.23762861],
       [0.        , 0.        ],
       [0.        , 0.02764711],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.00118497],
       [0.        , 0.23247805],
       [0.        , 0.32193223],
       [0.        , 0.        ],
       [0.        , 0.2675899 ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.03163347],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.00677812],
       [0.        , 0.19905475],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.21122071],
       [0.        , 0.        ],
       [0.